<a href="https://colab.research.google.com/github/farhanwew/FP-NLP/blob/main/Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

path = kagglehub.dataset_download("gvaldenebro/cancer-q-and-a-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cancer-q-and-a-dataset


In [2]:
! pip install chromadb

In [3]:
! pip install -U langchain-community

In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
os.listdir(path)

['growth_hormone_receptorQA.csv',
 'MedicalQuestionAnswering.csv',
 'Disease_Control_and_PreventionQA.csv',
 'Genetic_and_Rare_DiseasesQA.csv',
 'Diabetes_and_Digestive_and_Kidney_DiseasesQA.csv',
 'CancerQA.csv',
 'Neurological_Disorders_and_StrokeQA.csv',
 'Heart_Lung_and_BloodQA.csv',
 'SeniorHealthQA.csv',
 'OtherQA.csv']

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.schema import Document

# Use LangChain's wrapper for sentence-transformers
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-6-c30465359a7f>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

Nyoba similarity

In [7]:
sentence1 = "i like cat"
sentence2 = "i like kitten"
sentence3 = "i like tomato"

In [8]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [9]:
# similaar katanya
np.dot(embedding1, embedding2)

np.float64(0.8483984206512478)

In [10]:
# jauh konteks
np.dot(embedding1, embedding3)

np.float64(0.4965425707354012)

In [11]:
# jauh konteks
np.dot(embedding1, embedding3)

np.float64(0.4965425707354012)

In [12]:
data = pd.read_csv(path + "/CancerQA.csv")

In [13]:
data

,Question,Answer,topic,split
0,What is (are) Non-Small Cell Lung Cancer ?,Key Points\n - Non-small ce...,cancer,train
1,Who is at risk for Non-Small Cell Lung Cancer? ?,Smoking is the major risk factor for non-small...,cancer,train
2,What are the symptoms of Non-Small Cell Lung C...,Signs of non-small cell lung cancer include a ...,cancer,test
3,How to diagnose Non-Small Cell Lung Cancer ?,Tests that examine the lungs are used to detec...,cancer,train
4,What is the outlook for Non-Small Cell Lung Ca...,Certain factors affect prognosis (chance of re...,cancer,train
...,...,...,...,...
724,How to diagnose Adult Soft Tissue Sarcoma ?,Adult soft tissue sarcoma is diagnosed with a ...,cancer,test
725,What is the outlook for Adult Soft Tissue Sarc...,Certain factors affect treatment options and p...,cancer,test
726,What are the stages of Adult Soft Tissue Sarco...,Key Points\n - After adult ...,cancer,train
727,what research (or clinical trials) is being do...,Patients may want to think about taking part i...,cancer,train


In [14]:
from langchain.schema import Document
documents = []
for index, row in data.iterrows():
    answer_text = row['Answer']

    metadata = {
        'row_id': index
    }

    # Create a Document object
    doc = Document(
        page_content=answer_text,
        metadata=metadata
    )
    documents.append(doc)

## Vectorstores

In [15]:
from langchain.vectorstores import Chroma

In [16]:
persist_directory = 'docs/chroma/'

In [17]:
!rm -rf ./docs/chroma

In [18]:
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory=persist_directory
)

## Contoh  Search

### Similarity Search

In [19]:
question = "What is Non-Small Cell Lung Cancer"

In [20]:
# cari 3 top dokumen
docs = vectordb.similarity_search(question,k=3)

In [21]:
len(docs)

3

In [22]:
dir(docs)

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [23]:
dir(docs[0])

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_setattr_handl

In [24]:
docs[0].page_content

"Key Points\n                    - Non-small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.     - There are several types of non-small cell lung cancer.    - Smoking is the major risk factor for non-small cell lung cancer.    - Signs of non-small cell lung cancer include a cough that doesn't go away and shortness of breath.    - Tests that examine the lungs are used to detect (find), diagnose, and stage non-small cell lung cancer.    - Certain factors affect prognosis (chance of recovery) and treatment options.    - For most patients with non-small cell lung cancer, current treatments do not cure the cancer.\n                \n                \n                    Non-small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.\n                    The lungs are a pair of cone-shaped breathing organs in the chest. The lungs bring oxygen into the body as you breathe in. They release carbon dioxid

In [25]:
docs[1].page_content

'Key Points\n                    - Small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.    - There are two main types of small cell lung cancer.     - Smoking is the major risk factor for small cell lung cancer.    - Signs and symptoms of small cell lung cancer include coughing, shortness of breath, and chest pain.    - Tests and procedures that examine the lungs are used to detect (find), diagnose, and stage small cell lung cancer.    - Certain factors affect prognosis (chance of recovery) and treatment options.    - For most patients with small cell lung cancer, current treatments do not cure the cancer.\n                \n                \n                    Small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.\n                    The lungs are a pair of cone-shaped breathing organs that are found in the chest. The lungs bring oxygen into the body when you breathe in and take out car

In [26]:
docs[2].page_content

'Signs and symptoms of small cell lung cancer include coughing, shortness of breath, and chest pain. These and other signs and symptoms may be caused by small cell lung cancer or by other conditions. Check with your doctor if you have any of the following:         - Chest discomfort or pain.    - A cough that doesnt go away or gets worse over time.     - Trouble breathing.    - Wheezing.    -  Blood in sputum (mucus coughed up from the lungs).     - Hoarseness.    - Trouble swallowing.    - Loss of appetite.    - Weight loss for no known reason.    - Feeling very tired.     - Swelling in the face and/or veins in the neck.'

### MMR

In [27]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [28]:
docs_mmr[0].page_content

"Key Points\n                    - Non-small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.     - There are several types of non-small cell lung cancer.    - Smoking is the major risk factor for non-small cell lung cancer.    - Signs of non-small cell lung cancer include a cough that doesn't go away and shortness of breath.    - Tests that examine the lungs are used to detect (find), diagnose, and stage non-small cell lung cancer.    - Certain factors affect prognosis (chance of recovery) and treatment options.    - For most patients with non-small cell lung cancer, current treatments do not cure the cancer.\n                \n                \n                    Non-small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.\n                    The lungs are a pair of cone-shaped breathing organs in the chest. The lungs bring oxygen into the body as you breathe in. They release carbon dioxid

In [29]:
docs_mmr[1].page_content

'The cause of most childhood CNS germ cell tumors is not known.'

In [30]:
docs_mmr[2].page_content

'Signs and symptoms of small intestine cancer include unexplained weight loss and abdominal pain. These and other signs and symptoms may be caused by small intestine cancer or by other conditions. Check with your doctor if you have any of the following:          - Pain or cramps in the middle of the abdomen.    - Weight loss with no known reason.     - A lump in the abdomen.     -  Blood in the stool.'

## Chat

In [31]:
from google.colab import userdata
openR = userdata.get('OPEN_ROUTER')

In [32]:
from langchain_community.chat_models import ChatOpenAI
from typing import Optional
import os

class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(
        self,
        model_name: str,
        openai_api_key: Optional[str] = None,
        openai_api_base: str = "https://openrouter.ai/api/v1",
        **kwargs
    ):
        openai_api_key = openR
        super().__init__(
            openai_api_base=openai_api_base,
            openai_api_key=openai_api_key,
            model_name=model_name,
            **kwargs
        )

In [33]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenRouter(
    model_name="meta-llama/llama-4-maverick:free"
)
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
openrouter_chain = prompt | llm
print(openrouter_chain.invoke({"topic": "banana"}))

content="Why did the banana go to the doctor? Because it wasn't peeling well!" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 17, 'total_tokens': 33, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'meta-llama/llama-4-maverick:free', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-35d72da7-a8ce-434b-894e-32f8733ea238-0'


# Retrive

In [34]:
from langchain.chains import RetrievalQA

In [35]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [36]:
question = "What is Non-Small Cell Lung Cancer"

In [37]:
result = qa_chain({"query": question})

<ipython-input-37-ef85128c39b8>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [38]:
result["result"]

"Non-small cell lung cancer (NSCLC) is a disease in which malignant (cancer) cells form in the tissues of the lung. The lungs are a pair of cone-shaped breathing organs in the chest that bring oxygen into the body and release carbon dioxide. NSCLC is one of the two main types of lung cancer, the other being small cell lung cancer.\n\nThere are several types of NSCLC, including:\n\n1. Squamous cell carcinoma: Cancer that begins in squamous cells, which are thin, flat cells that look like fish scales.\n2. Large cell carcinoma: Cancer that may begin in several types of large cells.\n3. Adenocarcinoma: Cancer that begins in the cells that line the alveoli and make substances such as mucus.\n4. Other less common types, such as pleomorphic, carcinoid tumor, salivary gland carcinoma, and unclassified carcinoma.\n\nNSCLC can cause a range of signs and symptoms, including a cough that doesn't go away, shortness of breath, chest discomfort or pain, wheezing, and weight loss. However, sometimes N

In [39]:
question = 'Apa itu Program Studi Rekayasa Kecerdasan Artifisial secara singkat?'
result = qa_chain({"query": question})

In [40]:
result["result"]

'Maaf, saya tidak bisa menjawab pertanyaan tersebut karena tidak ada informasi yang relevan dalam teks yang diberikan. Teks tersebut membahas tentang Kaposi sarcoma, jenis-jenis pengobatan, dan percobaan klinis, bukan tentang Program Studi Rekayasa Kecerdasan Artifisial.'

In [41]:
rka = """
Pada program studi Rekayasa Kecerdasan Artifisial (artificial intelligence/AI), mahasiswa mempelajari cara mengembangkan sistem yang mampu meniru kemampuan manusia dalam berpikir, belajar, dan beradaptasi dengan lingkungan. Program studi ini mengembangkan keilmuan bidang ilmu komputer, khususnya bidang kecerdasan artifisial, untuk menciptakan solusi yang cerdas dan inovatif di bidang industri, pendidikan, kesehatan, dan lainnya. Contoh aplikasinya meliputi pengenalan objek pada gambar, chatbot, virtual assistant, mobil self-driving, dan sistem rekomendasi. Lulusan program studi ini diharapkan memiliki kompetensi dalam merancang, menganalisis, mengimplementasikan, dan menguji sistem kecerdasan artifisial yang efisien dan efektif.
"""

In [42]:
rka

'\nPada program studi Rekayasa Kecerdasan Artifisial (artificial intelligence/AI), mahasiswa mempelajari cara mengembangkan sistem yang mampu meniru kemampuan manusia dalam berpikir, belajar, dan beradaptasi dengan lingkungan. Program studi ini mengembangkan keilmuan bidang ilmu komputer, khususnya bidang kecerdasan artifisial, untuk menciptakan solusi yang cerdas dan inovatif di bidang industri, pendidikan, kesehatan, dan lainnya. Contoh aplikasinya meliputi pengenalan objek pada gambar, chatbot, virtual assistant, mobil self-driving, dan sistem rekomendasi. Lulusan program studi ini diharapkan memiliki kompetensi dalam merancang, menganalisis, mengimplementasikan, dan menguji sistem kecerdasan artifisial yang efisien dan efektif.\n'

In [43]:
new_documents = [
    Document(page_content=rka, metadata={"source": "tambahan"})
]

vectordb.add_documents(documents=new_documents)


['5199ab85-5120-47a5-b264-d48d5a84e394']

In [44]:
question = 'Apa itu Program Studi Rekayasa Kecerdasan Artifisial secara singkat?'
result = qa_chain({"query": question})

In [45]:
result["result"]

'Program Studi Rekayasa Kecerdasan Artifisial adalah program studi yang mempelajari cara mengembangkan sistem yang mampu meniru kemampuan manusia dalam berpikir, belajar, dan beradaptasi dengan lingkungan, serta menciptakan solusi cerdas dan inovatif di berbagai bidang.'